# Second Strategy 5 mins 1 trade per trend 

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.tradeID = None
        
        # NEW ---------------------------------------------------------
        # remembers whether we have already fired a ticket
        # in the still-running trend leg
        self.last_trend_seen   = 'NO'     # 'BU' / 'BE' / 'NO'
        self.trade_executed    = False    # True after 1st ticket of leg
        # -------------------------------------------------------------
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 1.7
        self.atr_period = 14
        self.atr_multiplier = 1.5 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = True
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base


        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size * 100000 # If we are trading AUDJPY, the value will be 0.01/100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  _calculate_atr  =============
        # ==============================================================
    def _calculate_atr(self, entry_point = None, trend = "NO", atr_period = None):
        
        atr_period = self.atr_period
        df = self.df
        atr_multiplier = self.atr_multiplier
        R = self.risk_reward_ratio
        
        current_idx = df.index.get_loc(entry_point)
        
        start_idx = max(0, current_idx - atr_period)
        
        df_slice = df.iloc[start_idx:current_idx + 1].copy()
        
        df_slice['H-L'] = df_slice['High'] - df_slice['Low']
        df_slice['H-PC'] = (df_slice['High'] - df_slice['Close'].shift(1)).abs()
        df_slice['L-PC'] = (df_slice['Low'] - df_slice['Close'].shift(1)).abs()
        df_slice['TR'] = df_slice[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        df_slice['ATR'] = df_slice['TR'].rolling(window=atr_period).mean()
        
        atr_value = df_slice['ATR'].iloc[-1]
        
        pip_size = atr_multiplier * atr_value
        
        current_close_price = df.loc[entry_point]["Close"]
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE":
            stop_loss_price = current_close_price + pip_size
            take_profit_price = current_close_price - R * pip_size
        else:  # bullish
            stop_loss_price = current_close_price - pip_size
            take_profit_price = current_close_price + R * pip_size
        
        return stop_loss_price, take_profit_price
        
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):
        
        # do not allow second ticket inside the same leg
        if self.trade_executed:
            return
        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # mark that we already tried to trade this leg
        self.trade_executed = True
        
        self.evt.consoleLog(
        f"{self.instrument} Order sent for trend: {trend} at price: {entry_price} | Stop: {stop} | Take: {take} | Volume: {volume}"
    )
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do
        
        df = self.df
        if self.enter_market_trade_side == "BU":          # long trade
            
            if (df.loc[ts]["EMA_5"] < df.loc[ts]["EMA_50"]) or (df.loc[ts]["MACD_fast"] < 0):
                    current_price = df.loc[ts]["Close"]
                    
                    self.evt.update_opened_order(tradeID=self.tradeID, tp=current_price, sl=current_price)
                    
                    # close trade
                    self.reset()
        elif self.enter_market_trade_side == "BE":          # long trade
            
            if (df.loc[ts]["EMA_5"] > df.loc[ts]["EMA_50"]) or (df.loc[ts]["MACD_fast"] > 0):
                    current_price = df.loc[ts]["Close"]
                    # close trade
                    self.evt.update_opened_order(tradeID=self.tradeID, tp=current_price, sl=current_price)
                    self.reset()
                    
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        """
       # safety – wait until bar exists
        if ts not in self.df.index:
            return
        """
        if self.position_open:
            self.monitor_exit(ts)
            return
        
        curr_data_at_ts = self.df.loc[ts]
        self.evt.consoleLog("curr_data_at_ts", curr_data_at_ts)
        if isinstance(curr_data_at_ts, pd.DataFrame):
            # Multiple rows for this timestamp, take the last one.
            curr_row_series = curr_data_at_ts.iloc[-1]
            
        else:
            # Single row for this timestamp, curr_data_at_ts is already a Series.
            curr_row_series = curr_data_at_ts
        
        current_trend = curr_row_series['trend']  # current_trend is now a scalar string ('BU', 'BE', 'NO')

        # ♦♦♦ trend changed?  →  allow exactly one new ticket ♦♦♦
        if (current_trend != self.last_trend_seen):
            self.trade_executed = False
            self.evt.consoleLog(f" Trend change from {self.last_trend_seen} to {current_trend}, allow 1 trade.")
            self.last_trend_seen = current_trend
            

        # nothing to do if we already traded in this leg
        if self.trade_executed or current_trend == "NO":
            return
        
        closes  = self.df["Close"]
        
        # ==============================================================
        # ========================  BEARISH  ============================
        # ==============================================================
        
        if (current_trend == "BE"):
            
            stop, take = self._calculate_atr(entry_point = ts, trend = current_trend)
                            
            # Logic to calculate lot_size
            entry_price = closes.loc[ts]
            stop_loss_pips = abs(stop - entry_price) / self.pip_size
            lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                
            
            # place selling order
            self.send_entry_order(ts, trend="BE",
                        entry_price=entry_price,
                        stop=stop, take=take,
                        volume=lot_size)
                        
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================                       
                                
        if (current_trend == "BU"):
            
            stop, take = self._calculate_atr(entry_point = ts, trend = current_trend)
                            
            # Logic to calculate lot_size
            entry_price = closes.loc[ts]
            stop_loss_pips = abs(stop - entry_price) / self.pip_size
            lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
            
            # place selling order
            self.send_entry_order(ts, trend="BU",
                        entry_price=entry_price,
                        stop=stop, take=take,
                        volume=lot_size)

    def reset(self):
        self.position_open = False
        self.stop_loss_price = None
        self.take_profit_price = None
        self.tradeID = None
        self.enter_market_trade_side = None

class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    MIN_BARS  = 50 
    MAX_BARS  = 300
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
    
                
        # ------------- 5-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
            "Datetime", "Open", "High", "Low", "Close",
            "Green", "Instrument", "EMA_5", "EMA_50", "EMA_12", 'EMA_26',"MACD_slow","MACD_fast", "trend"
        ])
        .astype({
            "Datetime": "datetime64[ns]",
            "Open":  NFLOAT,
            "High":  NFLOAT,
            "Low":   NFLOAT,
            "Close": NFLOAT,
            "Green": "category",
            "Instrument": "object",
            "EMA_5": NFLOAT,
            "EMA_50": NFLOAT,
            "EMA_12": NFLOAT,
            "EMA_26": NFLOAT,
            "MACD_slow": NFLOAT,
            "MACD_fast": NFLOAT,
            "trend": "category",
        })
        .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        ts = pd.to_datetime(snap["timestamp"])
        self.last_time = ts
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar = {
            "Datetime": ts,
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        bar = self._add_indicators_and_trend(bar)     # <– see next section
        
        if ts in self.data.index:
            self.data.loc[ts] = bar  # overwrite existing row
        else:
            self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime", drop=False)])  # append new bar
        
        
        if len(self.data) < self.MIN_BARS:
            return
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
            
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")
            # self.evt.consoleLog(f'{base_currency}USD, {fx_base}')
        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)
            # print out result to console log
            self.evt.consoleLog(f'USD{quote_currency}, {fx}')
        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,fx=fx,fx_base = fx_base, ts=self.last_time)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
                                 
    def _add_indicators_and_trend(self, bar: dict):
        n   = len(self.data)
        cl  = bar["Close"]
    
        if n == 0:
            ema5 = ema50 = ema12 = ema26 = cl
            macd_slow = macd_fast = 0.0
        else:
            prev = self.data.iloc[-1]
            ema5   = (1 - 2/6)   * prev["EMA_5"]  + 2/6   * cl
            ema50  = (1 - 2/51)  * prev["EMA_50"] + 2/51  * cl
            ema12  = (1 - 2/13)  * prev["EMA_12"] + 2/13  * cl
            ema26  = (1 - 2/27)  * prev["EMA_26"] + 2/27  * cl
            macd_slow = ema12 - ema26
            prev_macd_fast = prev["MACD_fast"] if n > 1 else 0.0
            macd_fast = (1 - 2/19) * prev_macd_fast + 2/19 * macd_slow
    
        bar["EMA_5"]     = ema5
        bar["EMA_50"]    = ema50
        bar["EMA_12"]    = ema12
        bar["EMA_26"]    = ema26
        bar["MACD_slow"] = macd_slow
        bar["MACD_fast"] = macd_fast
    
        if ema5 > ema50 and macd_fast > 0:
            bar["trend"] = "BU"
        elif ema5 < ema50 and macd_fast < 0:
            bar["trend"] = "BE"
        else:
            bar["trend"] = "NO"       
        
        return bar
        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
                fsm.tradeID   = of.tradeID 
                
                fsm.pending = None      # clear

        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                fsm.trade_executed = False      # ← NEW: unlock leg again
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass
